In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report,mean_squared_error, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedBaggingClassifier
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [2]:
# put the dataset which comes from short_dataset_feature_engineering.ipynb 
file_path = '/data/caysar9/results/final_short.csv'
df = pd.read_csv(file_path)

In [3]:
# Define features and target 
features = ['trigger_lack_physical_activity','trigger_physical_activity','trigger_poor_sleep','trigger_stress','sleep_duration_hours','sleep_duration_past_7_days','age','migraine_attacks_past7days','mean_migraine_duration_past7days','total_physical_activity','reported_anxiety', 'reported_depression']

target_intensity = 'painintensity'

X = df[features]
y = df[target_intensity]

# Handle missing values in the target variable
if y.isnull().sum() > 0:
    print(f"Missing values in target: {y.isnull().sum()}. Filling with median.")
    y = y.fillna(y.median())  # Fill NaN values with the median of the column


# Define bins for categorizing pain intensity (Low: 0-3, Medium: 4-7, High: 8-10)
bins = [0, 3, 7, 10]  # Define bin boundaries
labels = ['1', '2', '3']

# Bin the target variable
y_binned = pd.cut(y, bins=bins, labels=labels, include_lowest=True).astype(int)

# Frequency counts before splitting
print("Frequency counts for each class before sampling:")
print(y_binned.value_counts(sort=False))

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_binned, test_size=0.2, random_state=42, stratify=y_binned)

# Frequency counts before applying SMOTE
print("\nFrequency counts before SMOTE:")
print("Training set:")
print(y_train.value_counts(sort=False))
print("Test set:")
print(y_test.value_counts(sort=False))

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Frequency counts after applying SMOTE
print("\nFrequency counts after SMOTE:")
print(y_train_balanced.value_counts(sort=False))



Frequency counts for each class before sampling:
painintensity
2    9851
3    2818
1    1733
Name: count, dtype: int64

Frequency counts before SMOTE:
Training set:
painintensity
2    7881
1    1386
3    2254
Name: count, dtype: int64
Test set:
painintensity
2    1970
1     347
3     564
Name: count, dtype: int64

Frequency counts after SMOTE:
painintensity
2    7881
1    7881
3    7881
Name: count, dtype: int64


In [12]:
# Define hyperparameter distributions
param_distributions = {
    'C': uniform(0.01, 10),  # Regularization strength
    'max_iter': randint(100, 1000),  # Number of iterations
    'solver': ['lbfgs', 'newton-cg', 'sag'],  # Solvers that support multinomial
    'multi_class': ['multinomial']  # Multinomial setting for multi-class classification
}

# Initialize the Logistic Regression model
lr_model = LogisticRegression(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=lr_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Optimize based on AUC (One-vs-One for multi-class)
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/data/caysar9/venv-py311

Best Hyperparameters: {'C': 7.3299394181140505, 'max_iter': 800, 'multi_class': 'multinomial', 'solver': 'lbfgs'}

AUC Score (One-vs-One): 0.5487

Confusion Matrix:
[[141 129  77]
 [704 830 436]
 [153 244 167]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.14      0.41      0.21       347
      Medium       0.69      0.42      0.52      1970
        High       0.25      0.30      0.27       564

    accuracy                           0.40      2881
   macro avg       0.36      0.37      0.33      2881
weighted avg       0.54      0.40      0.44      2881



/data/caysar9/venv-py311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# Add a constant to the feature set for the intercept term
X_train_balanced_const = sm.add_constant(X_train_balanced)


# Fit a multinomial logistic regression model using statsmodels
logit_model = sm.MNLogit(y_train_balanced, X_train_balanced_const)
logit_results = logit_model.fit()

# Get the summary, including coefficients and p-values
print(logit_results.summary())

Optimization terminated successfully.
         Current function value: 1.038599
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:          painintensity   No. Observations:                23643
Model:                        MNLogit   Df Residuals:                    23617
Method:                           MLE   Df Model:                           24
Date:                Sat, 07 Dec 2024   Pseudo R-squ.:                 0.05463
Time:                        20:22:36   Log-Likelihood:                -24556.
converged:                       True   LL-Null:                       -25974.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 painintensity=2       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.0036      0.107     -9.41

In [14]:
# Define hyperparameter distributions
param_distributions = {
    'criterion': ['gini', 'entropy', 'log_loss'],  # Splitting criteria
    'max_depth': randint(3, 50),  # Maximum depth of the tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 10),  # Minimum number of samples in a leaf node
    'max_features': [None, 'sqrt', 'log2']  # Number of features considered for the best split
}

# Initialize the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=dt_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class problems
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 24, 'max_features': None, 'min_samples_leaf': 9, 'min_samples_split': 18}

AUC Score (One-vs-One): 0.5137

Confusion Matrix:
[[  64  197   86]
 [ 377 1157  436]
 [  93  338  133]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.12      0.18      0.15       347
      Medium       0.68      0.59      0.63      1970
        High       0.20      0.24      0.22       564

    accuracy                           0.47      2881
   macro avg       0.34      0.34      0.33      2881
weighted avg       0.52      0.47      0.49      2881



In [15]:
# Define hyperparameter distributions
param_distributions = {
    'n_estimators': randint(50, 300),  # Number of trees in the forest
    'max_depth': randint(5, 50),  # Maximum depth of each tree
    'min_samples_split': randint(2, 20),  # Minimum samples to split a node
    'min_samples_leaf': randint(1, 10),  # Minimum samples in a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider for the best split
    'bootstrap': [True, False]  # Whether bootstrap samples are used
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'bootstrap': False, 'max_depth': 34, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 253}

AUC Score (One-vs-One): 0.5665

Confusion Matrix:
[[  52  255   40]
 [ 189 1481  300]
 [  38  427   99]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.19      0.15      0.17       347
      Medium       0.68      0.75      0.72      1970
        High       0.23      0.18      0.20       564

    accuracy                           0.57      2881
   macro avg       0.37      0.36      0.36      2881
weighted avg       0.53      0.57      0.55      2881



In [16]:
# Get feature importance
feature_importance = best_model.feature_importances_
feature_names = X_train_balanced.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:\n", importance_df)



Feature Importance:
                              Feature  Importance
8   mean_migraine_duration_past7days    0.192158
6                                age    0.172768
5         sleep_duration_past_7_days    0.156918
4               sleep_duration_hours    0.154188
9            total_physical_activity    0.127205
7         migraine_attacks_past7days    0.121065
3                     trigger_stress    0.019474
2                 trigger_poor_sleep    0.017604
10                  reported_anxiety    0.015000
11               reported_depression    0.013614
0     trigger_lack_physical_activity    0.007838
1          trigger_physical_activity    0.002170


In [4]:
# Define hyperparameter distributions
param_distributions = {
    'n_estimators': randint(50, 300),  # Number of boosting stages
    'learning_rate': uniform(0.01, 0.3),  # Learning rate for boosting
    'max_depth': randint(3, 20),  # Maximum depth of each tree
    'min_samples_split': randint(2, 20),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': randint(1, 10),  # Minimum number of samples required in a leaf node
    'subsample': uniform(0.7, 0.3),  # Fraction of samples used for fitting each base learner
    'max_features': ['sqrt', 'log2', None]  # Number of features to consider for splits
}

# Initialize the Gradient Boosting model
gb_model = GradientBoostingClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gb_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Hyperparameters: {'learning_rate': 0.1913252137833452, 'max_depth': 17, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 14, 'n_estimators': 209, 'subsample': 0.8796596399465607}

AUC Score (One-vs-One): 0.5491

Confusion Matrix:
[[  39  277   31]
 [ 153 1555  262]
 [  29  440   95]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.18      0.11      0.14       347
      Medium       0.68      0.79      0.73      1970
        High       0.24      0.17      0.20       564

    accuracy                           0.59      2881
   macro avg       0.37      0.36      0.36      2881
weighted avg       0.54      0.59      0.56      2881



In [5]:
import joblib
# Save the gradient boosting trained model to a file to use it in a web app
model_filename = 'gb_model_short_dataset_pain_intensity.pkl'
joblib.dump(best_model, model_filename)
print(f"Model saved to {model_filename}")

Model saved to gb_model_short_dataset_pain_intensity.pkl


In [18]:
# Get feature importance
feature_importance = best_model.feature_importances_
feature_names = X_train_balanced.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance}).sort_values(by='Importance', ascending=False)
print("\nFeature Importance:\n", importance_df)



Feature Importance:
                              Feature  Importance
8   mean_migraine_duration_past7days    0.200612
6                                age    0.175545
9            total_physical_activity    0.143004
5         sleep_duration_past_7_days    0.142228
4               sleep_duration_hours    0.137910
7         migraine_attacks_past7days    0.131648
3                     trigger_stress    0.025561
2                 trigger_poor_sleep    0.015547
11               reported_depression    0.012284
10                  reported_anxiety    0.009569
0     trigger_lack_physical_activity    0.004717
1          trigger_physical_activity    0.001376


In [19]:
# Define hyperparameter distributions
param_distributions = {
    'n_neighbors': randint(1, 50),  # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weighting strategies
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Distance metrics
    'p': randint(1, 3)  # distance parameter 
}

# Initialize the K-Nearest Neighbors model
knn_model = KNeighborsClassifier()

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn_model,
    param_distributions=param_distributions,
    n_iter=20,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Evaluate based on AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Show progress
    n_jobs=-1,  # Use all available cores
    random_state=42  # For reproducibility
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_balanced, y_train_balanced)

# Get the best model and hyperparameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Train the best model on the balanced training set
best_model.fit(X_train_balanced, y_train_balanced)

# Make predictions and evaluate
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)  # Probabilities for each class

# Calculate AUC
auc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Low", "Medium", "High"]))


Fitting 3 folds for each of 20 candidates, totalling 60 fits


Best Hyperparameters: {'metric': 'manhattan', 'n_neighbors': 6, 'p': 2, 'weights': 'distance'}

AUC Score (One-vs-One): 0.5530

Confusion Matrix:
[[128 123  96]
 [583 815 572]
 [123 228 213]]

Classification Report:
              precision    recall  f1-score   support

         Low       0.15      0.37      0.22       347
      Medium       0.70      0.41      0.52      1970
        High       0.24      0.38      0.29       564

    accuracy                           0.40      2881
   macro avg       0.36      0.39      0.34      2881
weighted avg       0.54      0.40      0.44      2881



In [20]:
'''from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Define features and target
features = ['trigger_lack_physical_activity','trigger_physical_activity','trigger_poor_sleep','trigger_stress','sleep_duration_hours','sleep_duration_past_7_days','age','migraine_attacks_past7days','mean_migraine_duration_past7days','total_physical_activity','reported_anxiety', 'reported_depression']

target_intensity = 'painintensity'

X = df[features]
y = df[target_intensity]

# Handle missing values in the target variable
if y.isnull().sum() > 0:
    print(f"Missing values in target: {y.isnull().sum()}. Filling with median.")
    y = y.fillna(y.median())

# Define bins for categorizing pain intensity (Low: 0-3, Medium: 4-7, High: 8-10)
bins = [0, 3, 7, 10]
labels = [0, 1, 2]  # Integers for Low, Medium, High
y_binned = pd.cut(y, bins=bins, labels=labels, include_lowest=True).astype(int)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binned, test_size=0.2, random_state=42, stratify=y_binned)

# Apply SMOTE to the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Define a function to create the neural network model
def create_model(learning_rate=0.001, neurons=[64, 32, 16]):
    model = Sequential()
    model.add(Dense(neurons[0], activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dense(neurons[1], activation='relu'))
    model.add(Dense(neurons[2], activation='relu'))
    model.add(Dense(3, activation='softmax'))  # 3 classes: Low, Medium, High
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model for RandomizedSearchCV
keras_clf = KerasClassifier(model=create_model, verbose=0)

# Define hyperparameter distributions
param_distributions = {
    "model__learning_rate": [0.001, 0.01, 0.1],
    "model__neurons": [[64, 32, 16], [128, 64, 32], [32, 16, 8]],
    "batch_size": [16, 32, 64],
    "epochs": [20, 50, 100]
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=keras_clf,
    param_distributions=param_distributions,
    n_iter=10,  # Number of random combinations to try
    scoring='roc_auc_ovo',  # Optimize AUC for multi-class classification
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available cores
    random_state=42
)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train_scaled, y_train_smote)

# Get the best model and parameters
best_model = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate the model
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)

# Calculate AUC
auc_score = roc_auc_score(pd.get_dummies(y_test), y_pred_proba, multi_class='ovo')
print(f"\nAUC Score (One-vs-One): {auc_score:.4f}")

# Confusion Matrix and Classification Report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

target_names = ["Low", "Medium", "High"]
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=target_names))

'''

' \nfrom sklearn.metrics import roc_auc_score, confusion_matrix, classification_report\nfrom sklearn.model_selection import RandomizedSearchCV, train_test_split\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Dense\nfrom tensorflow.keras.optimizers import Adam\nfrom scikeras.wrappers import KerasClassifier\nfrom imblearn.over_sampling import SMOTE\nfrom sklearn.preprocessing import StandardScaler\nimport numpy as np\n\n# Define features and target\nfeatures = [\'trigger_lack_physical_activity\',\'trigger_physical_activity\',\'trigger_poor_sleep\',\'trigger_stress\',\'sleep_duration_hours\',\'sleep_duration_past_7_days\',\'age\',\'migraine_attacks_past7days\',\'mean_migraine_duration_past7days\',\'total_physical_activity\',\'reported_anxiety\', \'reported_depression\']\n\ntarget = \'attack_duration_hours\'\n\nX = df[features]\ny = df[target]\n\n# Define bins for migraine duration (Low: 4-9, Medium: 10-22, High: 23-72 hours)\nbins = [4, 9, 22, 72]\ny_